In [154]:
import os
import time
import openai

In [3]:
dataDirectory = "Dataset/ijcnlp_dailydialog"
textFile = dataDirectory + "/dialogues_text.txt"
emotionFile = dataDirectory + "/dialogues_emotion.txt"


In [17]:
def readLinebyLine(filePath,maxLine=float('inf')):
    currLine=0
    with open(filePath, 'r') as file:
    # Loop through each line in the file
        for line in file:
            if(currLine==maxLine):
                return
            # Process each line here
            print(line.strip())  # Print the line after stripping whitespace (e.g., newline characters)
            currLine+=1

In [ ]:
readSpecificConversations()

In [34]:
def extractEmotionConversations(emotionFile, textFile, label):
    conversationNumbers = set()
    lineNumber = 0
    count=0
    with open(emotionFile, 'r') as file:
    # Loop through each line in the file
        for line in file:

            if(label in line):
                conversationNumbers.add(lineNumber)
                count+=line.count(label)
            lineNumber+=1
    #print(len(conversationNumbers), count)
    return conversationNumbers

In [36]:
angryConversations = extractEmotionConversations(emotionFile,textFile,'1')
print(angryConversations)

{1, 2, 10251, 12301, 15, 32, 2084, 4132, 4134, 10281, 4143, 12337, 2100, 2101, 2102, 2106, 2111, 2113, 4161, 4165, 6214, 76, 2126, 4179, 4182, 8278, 8283, 2140, 4188, 95, 96, 97, 2148, 4196, 12395, 4205, 4208, 4211, 12403, 2168, 4216, 4217, 12408, 4220, 8318, 8321, 4227, 4228, 8324, 4231, 136, 12425, 4237, 4243, 10388, 10389, 10390, 10392, 10393, 155, 4251, 157, 10395, 10396, 4256, 4257, 4259, 4263, 6139, 2218, 10413, 12653, 2223, 2224, 4273, 178, 10415, 2228, 10416, 2230, 4278, 4280, 186, 4283, 2236, 2237, 10429, 2239, 2241, 2243, 4308, 2262, 4310, 4314, 222, 4320, 4321, 2275, 4327, 2280, 8428, 2285, 12530, 4348, 2302, 4354, 8452, 4362, 2324, 10517, 4377, 4379, 4387, 295, 296, 6440, 8487, 300, 4398, 306, 311, 8503, 4414, 10564, 2374, 327, 4423, 8521, 4426, 4427, 2380, 8522, 8526, 2383, 2384, 4431, 338, 2387, 8534, 2391, 351, 2402, 4451, 12642, 357, 364, 2413, 4460, 2415, 368, 2416, 370, 4465, 12284, 12652, 4470, 12656, 4472, 2427, 380, 4477, 12674, 2435, 12677, 12678, 12679, 392, 1268

In [191]:
prompts = [
    '\nState if the tone/intent of the last message in the conversation above is negative or not this format: "Y" for yes and "N" for no.'
]

def attachLabels(line):
    rawConvo=line.split('__eou__')
    proConvo = [item for item in rawConvo if item != '\n']
    for i in range(len(proConvo)):
        if (i%2 == 1):
            proConvo[i] = "Person 2: " + proConvo[i]
        else:
            proConvo[i] = "Person 1: " + proConvo[i]
    return proConvo

def prepareString(withLabels,currentIndex):
    currString = ''
    for i in range(0,currentIndex+1):
        currString = currString + withLabels[i] + "'\n"
    return currString

def preparePrompt(stringGPT, message):
    finalPrompt = stringGPT
    return finalPrompt

def callGPT(finalPrompt):
    #openai.api_type = "azure"
    #print("Prompt:\n", finalPrompt,'\n\n')
    openai.api_base = "REMOVED"
    openai.api_key = 'REMOVED'
    openai.api_version = "2023-05-15"
    response = openai.ChatCompletion.create(
        engine="GPT4",
        messages=[

            {"role": "user", "content": finalPrompt}
        ]
    )

    if (response['choices'][0]['message']['content'] == 'N'):
        return '0'
    else:
        return '1'
    
def writeOutput(outputGPT, promptChosen):

    finalOutput = promptChosen + "\n" + outputGPT
    file_path = "GPToutput.txt"

    # Open the file in write mode ('w')
    # This will create the file if it doesn't exist or completely overwrite it if it does
    with open(file_path, 'w') as file:
        file.write(finalOutput)

    # The file is automatically closed when the 'with' block exits




def readSpecificConversations(filePath,conversationNumbers,promptChosen, max=float('inf'), startFrom=0):
    currLine=0
    currOutput=''
    currCount=0
    with open(filePath, 'r') as file:
    # Loop through each line in the file
        for line in file:
            if(currLine in conversationNumbers and currLine>=startFrom):
                withLabels = attachLabels(line)
                for i in range(len(withLabels)):
                    convoGPT = prepareString(withLabels,i)
                    finalPrompt = preparePrompt(convoGPT + promptChosen, withLabels[i])
                    outputGPT=""
                    GPTdone = False
                    while(GPTdone==False):
                        try:  
                            outputGPT = callGPT(finalPrompt)
                            GPTdone=True
                        except Exception as e:
                            #prints index
                            #line number is index + 1
                            print("Conversation index: ", currLine)
                            if('response was filtered due to the prompt triggering' in str(e)):
                                print("Skipped by GPT.")    
                                outputGPT = "S"
                                GPTdone=True
                            else:
                                print(e,"END")
                                time.sleep(8)
                    currOutput = currOutput +" " + outputGPT
                currOutput = currOutput + "\n"
                currCount+=1
                if(currCount==max):
                    writeOutput(currOutput, promptChosen)
                    return '\n'.join([line.lstrip() for line in currOutput.splitlines()])

            
            currLine+=1
    writeOutput(currOutput, promptChosen)
    return '\n'.join([line.lstrip() for line in currOutput.splitlines()])


In [192]:
labelsGPT = readSpecificConversations(textFile,angryConversations,promptChosen=prompts[0])
print(labelsGPT)

Conversation index:  15
Requests to the Creates a completion for the chat message Operation under Azure OpenAI API version 2023-05-15 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 8 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit. END
Conversation index:  95
Requests to the Creates a completion for the chat message Operation under Azure OpenAI API version 2023-05-15 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 7 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit. END
Conversation index:  97
Requests to the Creates a completion for the chat message Operation under Azure OpenAI API version 2023-05-15 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 7 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like t